# Lab

In [1]:
import psutil

In [5]:
t = psutil.cpu_times()
t.user

scputimes(user=5783.53, nice=0.0, system=1778.19, idle=1197688.82, iowait=32.5, irq=0.0, softirq=175.48, steal=0.0, guest=0.0, guest_nice=0.0)

In [6]:
psutil.cpu_percent()

0.1

In [7]:
vmem = psutil.virtual_memory()
vmem.total

108014194688

In [11]:
psutil.disk_usage('/')

sdiskusage(total=527371075584, used=64355737600, free=438774947840, percent=12.8)

In [10]:
psutil.net_io_counters()

snetio(bytes_sent=512275269, bytes_recv=519558785, packets_sent=661275, packets_recv=668028, errin=0, errout=0, dropin=0, dropout=0)

In [16]:
psutil.disk_io_counters(perdisk=False, nowrap=True)

sdiskio(read_count=11481, write_count=18775, read_bytes=391936000, write_bytes=4400111616, read_time=7677, write_time=40004, read_merged_count=23695, write_merged_count=8213, busy_time=104600)

In [18]:
psutil.net_io_counters(pernic=False, nowrap=True)

snetio(bytes_sent=99162918, bytes_recv=100139078, packets_sent=101551, packets_recv=105423, errin=0, errout=0, dropin=0, dropout=0)

In [19]:
psutil.virtual_memory()

svmem(total=108014194688, available=106258292736, percent=1.6, used=849149952, free=106655449088, active=242708480, inactive=831938560, buffers=154234880, cached=355360768, shared=69632, slab=96374784)

In [2]:
108,014,194,688
FILEPATH = "data/sources/movielens25m/raw/ml-25m/ratings.csv"
ratings = IOService.read(FILEPATH)
scputimes(user=87.77, nice=0.0, system=97.39, idle=244789.86, iowait=4.92, irq=0.0, softirq=5.57, steal=0.0, guest=0.0, guest_nice=0.0)

In [3]:
ratings.head()
ratings.info()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


In [4]:
FILEPATH = "data/sources/movielens25m/raw/ml-25m/ratings.pkl"
IOService.write(filepath=FILEPATH, data=ratings)
r = IOService.read(FILEPATH)


In [5]:
r.head()
r.info()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


In [15]:
import threading
import time
import logging
class Profile:
    def __init__(self):
        self.i = 0
    def snapshot(self):
        self.i += 1        




In [23]:

def profiler(func): 
    active = threading.Lock() 
    profile = Profile()
     
    def profiling(profiler): 
        while active.locked(): 
            print("Inside loop")
            profiler.snapshot()
            time.sleep(1) 
             
    def wrapper(*args, **kwargs): 
        print("Inside wrapper")
        t = threading.Thread(target=profiling,args=(profile,)) 
        print("Started thread")
        active.acquire() 
        print("Acquired lock")
        t.start() 
        print("Started Thread")
        res = func(*args, **kwargs) 
        print("Launched wrapped function")
        active.release() 
        print("Releaseed the lock")
        print(profile.i)
        print("Printed i")
        return res 
     
    return wrapper 

In [24]:
@profiler 
def test(): 
    time.sleep(3) 
test()

Inside wrapper
Started thread
Acquired lock
Inside loop
Started Thread
Inside loop
Inside loop
Launched wrapped function
Releaseed the lock
3
Printed i
